In [0]:
import sys
from pyspark.sql.functions import col

workspace = '/Workspace/Shared/eixo5_grupo2_20252/projeto/processamento'
sys.path.insert(0, workspace)

import schemas.schemas_silver as ss

In [0]:
tabelas = spark.sql('SHOW TABLES IN filmes.bronze').select('tableName').collect()
catalogo = 'filmes'
schema = 'silver'

for t in tabelas:
    tabela = t['tableName']

    df = spark.table(f'{catalogo}.bronze.{tabela}')
    schema_silver = eval(f"ss.{tabela}.jsonValue()")

    for v, k in schema_silver.items():
        if v == 'fields':
            for i in k:
                df = df.withColumn(i['name'], col(i['name']).cast(i['type']))

    (df.write
        .mode('overwrite')
        .format('delta')
        .option('optimizeWrite', 'true')
        .option('autoCompact', 'true')
        .option('mergeSchema', 'true')
        .saveAsTable(f'{catalogo}.{schema}.{tabela}')
    )

In [0]:
df.display()